In [34]:
from pdfextractor import extractor
import pdfextractor
import pickle
import jieba
import pandas as pd
import re
import requests
import operator

In [136]:
class jmdPDFMap(object):
    """
    根据pdf生成文章，并计算推荐的准确率和召回率
    意图：导语，业绩变动原因，主营业务$业务进展，未来计划
    """
    def __init__(self):
        self.labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
        self.labelmap = {
                         '导语': ['意图_企业业绩_现状偏好', '意图_企业业绩_现状偏坏', '导语', '意图_企业基本信息介绍_企业基本信息介绍'],
                         '业绩变动原因': ['意图_企业业绩_业绩变动原因'],
                         '主营业务$业务进展': ['意图_企业业务_业务介绍', '意图_企业业务_业务规模', '意图_企业业务_业务规划'],
                         '未来计划': ['意图_企业业务_业务前景', '意图_企业业绩_前景偏好','意图_企业业绩_前景偏好'],
                         'other': ['意图_企业活动_公司公告信息', '意图_企业活动_投资活动', '意图_企业活动_人事活动', '意图_企业活动_融资活动', '其他', '意图_评论_风险提示','意图_评论_投资建议', '意图_评论_盈利预测']
                        }
        self.labelmapR = {
                          '意图_企业业绩_现状偏好': '导语', '意图_企业业绩_现状偏坏': '导语', '导语': '导语', '意图_企业基本信息介绍_企业基本信息介绍': '导语', 
                          '意图_企业业绩_业绩变动原因': '业绩变动原因',
                          '意图_企业业务_业务介绍': '主营业务$业务进展', '意图_企业业务_业务规模': '主营业务$业务进展', '意图_企业业务_业务规划': '主营业务$业务进展', 
                          '意图_企业业务_业务前景': '未来计划', '意图_企业业绩_前景偏好': '未来计划', '意图_企业业绩_前景偏坏': '未来计划',
                          '意图_企业活动_公司公告信息': 'other', '意图_企业活动_投资活动': 'other', '意图_企业活动_人事活动': 'other', '意图_企业活动_融资活动': 'other', '其他': 'other', '意图_评论_风险提示': 'other', '意图_评论_投资建议': 'other', '意图_评论_盈利预测':'other'
                         }
        self.prop = [0.2, 0.2, 0.4, 0.3, 0]
        self.K = 0
        #每个意图对应的句子或段落数量
        self.num = {}
        self.recommendList = []
        
        
    def computePR(self, article, textList = None, pdfurl = None, mode = 'para'):
        """
        pdfurl和textList为内容来源，必须有且仅有一个为None
        mode 必须指明
        """
        if ((pdfurl is None) and (textList is None)) or ((not pdfurl is None) and (not textList is None)):
            raise("pdfurl and textList can't be None or be 'NOT NONE' Simultaneously ")
        intentList = []
        #article_sen = self.get_article_sentence(article)
        article_sen = article
        article_len = len(article_sen)
        if article_len == 0:
            return -1, -1
        print 'article length: ' + str(article_len)
        if not pdfurl is None:
            if mode == 'para':
                try:
                    pdftext = self.extractPDF(pdfurl, 0.8, ['para'])
                except BaseException:
                    return -1, -1
            else:
                try:
                    pdftext = self.extractPDF(pdfurl, 0.8, ['text'])
                except BaseException:
                    return -1, -1
        else:
            pdftext = textList
        for t in pdftext:
            intentList.append(self.getIntent(t))
        if mode == 'para':
            self.K = int(len(article_sen) * 0.5) + 1
        else:
            self.K = int(len(article_sen) * 1.5) + 1
        for l in range(len(self.labels)):
            if l == len(self.labels) - 1:
                self.num[self.labels[l]] = 0
            else:
                self.num[self.labels[l]] = int(self.K * self.prop[l]) + 1
        self.recommendList = self.getTopKintent(intentList)
        rec_len = len(self.recommendList)
        if rec_len == 0:
            return -1, -1
        print 'recommend list length: ' + str(rec_len)
        P_dic = {}
        Pcount = 0
        Rcount = 0
        for r in self.recommendList:
            P_dic[r] = False
        for sen in article_sen:
            for rec in self.recommendList:
                if self.MatchTwoText(sen, rec):
                    Rcount += 1
                    if P_dic[rec] is False:
                        P_dic[rec] = True
                        Pcount += 1
                    break
        P = Pcount * 1.0 / rec_len
        R = Rcount * 1.0 / article_len
        print 'Pcount: ' + str(Pcount)
        print 'Rcount: ' + str(Rcount)
        return P, R
    
    def getRec(self):
        return self.recommendList
        
    def getTopKintent(self, intentlist):
        intent = {}
        for l in self.labels:
            intent[l] = {}
        
        for i in intentlist:
            key = i['intent']
            intent[key][i['content']] = i['prob']
        result = []
        for l in self.labels:
            intent_l = intent[l]
            intent[l] = sorted(intent_l.items(), key=operator.itemgetter(1))
        print "各个段落（句子）的预测意图分布为："
        for i in intent:
            print i + "  " + str(len(intent[i]))
        for l in intent.keys():
            if self.num[l] == 0:
                continue
            result.extend([d[0] for d in intent[l][-self.num[l]:]])
        return result
        
    def get_article_sentence(self, ar):
        """
        given an article, return the list of sentences
        """
        textList = ar.encode('utf-8').split('。') 
        textList = [t.replace('\n', '') for t in textList]
        textList = filter(lambda x: x != '', textList)
        return textList

    #percent: 取pdf的前多少页
    def extractPDF(self, pdfPath, percent, source = ['table', 'text', 'para']):
        savePath = '/home/zhuyuhe/mydata/jmd/percent/'
        ex = extractor.StockFinanceExtractor(pdfPath, percent)

        nodes = ex.get_pdf_objects(filter_headline=True)

        paraList = []
        textList = []
        textNodes = []
        tableList = []
        tableNodes = []
        senList = []
        for node in nodes:
            if isinstance(node.obj, pdfextractor.pdftree.VirtualNode):
                continue
            if isinstance(node.obj, pdfextractor.page.Table):
                tableNodes.append(node)
            elif  not isinstance(node.obj, pdfextractor.page.Catalog):
                textNodes.append(node)
        if 'table' in source:
            tableList = self.get_node_texts(tableNodes)
            tableList = [t.replace('|', '').split('\n') for t in tableList]
            #二维数组转化为一维
            tableList = sum(tableList,[])
        textList = self.get_node_texts(textNodes)
        if 'para' in source:
            paraList = [t.replace('\n', '') for t in textList]
            paraList = filter(lambda x: x != '', paraList)
        elif 'text' in source:
            senList = sum([t.encode('utf-8').split('。') for t in textList],[])
            senList = [t.replace('\n', '') for t in senList]
            senList = filter(lambda x: x != '', senList)

        pdf = []
    #     pdf.append(textList)
    #     pdf.append(tableList)
    #     pdf = sum(pdf, [])
#         print pdf
        pdf.extend(senList)
#         print 'after extent senList: ' + str(pdf) 
        pdf.extend(tableList)
#         print 'after extent tableList: ' + str(pdf) 
        pdf.extend(paraList)
#         print 'after extent paraList: ' + str(pdf) 
        print 'pdflist length: ' + str(len(pdf))

        return pdf
    
    def get_node_texts(self, pdf_nodes):
        '''
        Given a list of nodes, get the texts of these nodes
        '''
        texts = []
        for node in pdf_nodes:
            text = node.obj.get_text()
            if len(text) <= 25:
                continue
            if u'□' in text or u'√' in text:
                continue
            texts.append(text)
        return texts
    
    def getIntent(self, text):
        """
        return intent of the given text
        result: {'content': __, 'intent': __ , 'prob': __}
        """
        result = {}
        query = requests.post("https://surreal.aidigger.com/api/v1/classification/intent/finance/0",json={ "query": text}).json()
        predict = query['predict']
        
        result['content'] = text
        result['intent'], result['prob'] = self.intentMap(predict)
        return result
    
    def intentMap(self, predict):
        """
        将现有意图映射到芥末堆所给的意图上去, 并将权重求和
        """
        result = {}
        for l in self.labels:
            result[l] = 0
        for p in predict:
            result[self.labelmapR[p['name'].encode('utf-8')]] += p['prob']
        maxP = max(result.values())
        intent = result.keys()[result.values().index(maxP)]
        return intent, maxP
        
        
    def MatchTwoText(self, t1, t2, threshold = 0.5):
        """
        检查两个句子是否匹配
        """
        ct1 = ' '.join(jieba.cut(t1)).split()
        ct2 = ' '.join(jieba.cut(t2)).split()
        commonL = set(ct1).intersection(set(ct2)).__len__()
        if commonL * 1.0 / set(ct1).__len__() > threshold:
            return True
        return False

    def splitByPeriod(self, P):
        """
        split paragraph to sentences
        """
        senList = sum([t.encode('utf-8').split('。') for t in P],[])
        senList = [t.replace('\n', '') for t in senList]
        senList = filter(lambda x: x != '', senList)
        return senList

In [ ]:
#结果为-1代表mapcontent长度为0
a = jmdPDFMap()
jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'rb'))
url = []
for p in jmdata_map.keys():
    if jmdata_map[p]['mapcontent'].__len__() == 0:
        url.append(p)
for u in url:
    del jmdata_map[u]
P_para_map = []
R_para_map = []
P_sen_map = []
R_sen_map = []
for key in jmdata_map.keys():
    print key  
    print '---------PARA-----------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], textList=jmdata_map[key]['para'], mode = 'para')
    P_para_map.append(P)
    R_para_map.append(R)
    jmdata_map[key]['recPara'] = a.getRec()
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------SEN-------------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], textList=jmdata_map[key]['sen'], mode = 'sen')
    P_sen_map.append(P)
    R_sen_map.append(R)
    jmdata_map[key]['recSen'] = a.getRec()
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '*******************************************************'
    

http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
---------PARA-----------
article length: 9


In [111]:
print P_para_map
print R_para_map
print P_sen_map
print R_sen_map

[0.2222222222222222, 0.18181818181818182, 0.14285714285714285, 0.3333333333333333, 0.0, 0.0, 0.0, 0.5, 0.16666666666666666, 0.375, 0.2, 0.375, 0.375, 0.2727272727272727, 0.0, 0.3, 0.0, 0.125, 0.2857142857142857]
[0.4444444444444444, 0.25, 0.14285714285714285, 0.3333333333333333, -1, 0.0, 0.0, 0.0, -1, 0.7142857142857143, 0.16666666666666666, -1, 0.6666666666666666, 0.6, 0.5, 0.5555555555555556, 0.2, 0.0, 0.5384615384615384, 0.0, 0.1, 0.18181818181818182]
[0.0625, 0.1, 0.0, 0.3333333333333333, -1, 0.0, 0.1111111111111111, 0.0, -1, 0.35714285714285715, 0.0, -1, 0.25, 0.2727272727272727, 0.13333333333333333, 0.25, 0.19230769230769232, 0.0, 0.25, 0.08333333333333333, 0.11764705882352941]
[0.1111111111111111, 0.16666666666666666, 0.0, 0.4444444444444444, -1, 0.0, 0.75, 0.0, -1, 0.7142857142857143, 0.0, -1, 0.5555555555555556, 0.6, 0.375, 0.4444444444444444, 0.4, 0.0, 0.5384615384615384, 0.14285714285714285, 0.2]


In [97]:
import numpy as np
print 'p_para mean: ' + str(np.mean(filter(lambda x: x!=-1, P_para_map)))
print 'r_para mean: ' + str(np.mean(filter(lambda x: x!=-1, R_para_map)))
print 'p_sen mean: ' + str(np.mean(filter(lambda x: x!=-1, P_sen_map)))
print 'r_sen mean: ' + str(np.mean(filter(lambda x: x!=-1, R_sen_map)))

print "after filter 0:"
print 'p_para mean: ' + str(np.mean(filter(lambda x: x!=0 and x!=-1,P_para_map)))
print 'r_para mean: ' + str(np.mean(filter(lambda x: x!=0 and x!=-1,R_para_map)))
print 'p_sen mean: ' + str(np.mean(filter(lambda x: x!=0 and x!=-1,P_sen_map)))
print 'r_sen mean: ' + str(np.mean(filter(lambda x: x!=0 and x!=-1,R_sen_map)))

p_para mean: 0.202912584492
r_para mean: 0.283899433899
p_sen mean: 0.139635332895
r_sen mean: 0.302379256546
after filter 0:
p_para mean: 0.275381364667
r_para mean: 0.385292088864
p_sen mean: 0.193341230162
r_sen mean: 0.418678970602


In [ ]:
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
P_para = []
R_para = []
P_sen = []
R_sen = []
i = 0
for jmap in jmdata_mapping:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1
#print a.getIntent('前三季度利润为0，同比下降5%')['intent']

# jmap = jmd_mapping[0]
# print a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')

In [ ]:
a = jmdPDFMap()
for jmap in jmdata_mapping[14:]:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1

In [ ]:
print 'P_para: ' + str(P_para)
print 'R_para: ' + str(R_para)
print 'P_sen: ' + str(P_sen)
print 'R_sen: ' + str(R_sen) 
print len(P_para)

In [ ]:
i = 0
jmdata_map = {}
P_para2 = []
P_sen2 = []
R_para2 = []
R_sen2 = []
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
for jmap in jmdata_mapping:
    key = jmap['pdf_url']
    try:
        para = a.extractPDF(jmap['pdf_url'], 0.5, 'para')
    except BaseException:
        continue
    jmdata_map[key] = {}
    content = a.get_article_sentence(jmap['content'])
    sen = a.splitByPeriod(para)
    jmdata_map[key]['content'] = content
    jmdata_map[key]['para'] = para
    jmdata_map[key]['sen'] = sen
    print '---' + key + ' done' + '----'
    i+=1


Pdfminer Parsed time: 39s
Construct Pdf Tree time: 0s
pdflist length: 182
---http://www.cninfo.com.cn/finalpage/2017-06-20/1203635303.PDF done----
Download pdf time: 7s
download http status code: 200
Pdfminer Parsed time: 36s
Construct Pdf Tree time: 0s
pdflist length: 166
---http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF done----
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 17s
Construct Pdf Tree time: 0s
pdflist length: 46
---http://www.cninfo.com.cn/finalpage/2017-07-26/1203737887.PDF done----
Download pdf time: 6s
download http status code: 200
Pdfminer Parsed time: 35s
Construct Pdf Tree time: 0s
pdflist length: 86
---http://disclosure.szse.cn/finalpage/2017-08-25/1203872097.PDF done----
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 6s
Construct Pdf Tree time: 0s
pdflist length: 11
---http://disclosure.szse.cn/finalpage/2016-10-18/1202765393.PDF done----
Download pdf time: 1s
download http status code: 200
P

In [32]:

print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['content'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['mapcontent'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['para'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['sen'])

18
12
145
255


In [ ]:
Ppara = P_para2
Rpara = R_para2
Psen = P_sen2
R_sen = R_sen2

In [108]:
text = '公司负责人黄元忠、主管会计工作负责人孙晓玲及会计机构负责人(会计主管人员)全丽声明：保证季度报告中财务报告的真实、完整。'
query = requests.post("https://surreal.aidigger.com/api/v1/classification/intent/finance/0",json={ "query": text}).json()
predict = query['predict']
prob = [x['prob'] for x in predict]
i = max(prob)
intent = predict[prob.index(i)]['name']
print intent

其他


In [ ]:
p_para = [0.14, 0.2,0.22,1,0,0,0.5,0.125,0.27,0.5,0.28,0.1,0.33,0.09]
r_para = [0.24,0.09,0.19,0,0.2,0,0.67,0.14,0.12,0.31,0.36,0.125,0.14,0.04]
p_sen = [0,0,0,0,0.14,0,0.19,0.16,0.03,0,0.42,0.03,0.07,0.5]
r_sen = [0,0,0,0,0.2,0,0.25,0.23,0.04,0,0.36,0.04,0.25,0.27]

In [24]:
import numpy as np
print 'p_para mean: ' + str(np.mean(p_para))
print 'r_para mean: ' + str(np.mean(r_para))
print 'p_sen mean: ' + str(np.mean(p_sen))
print 'r_sen mean: ' + str(np.mean(r_sen))

print "after filter:"
print 'p_para mean: ' + str(np.mean(filter(lambda x: x!=0,p_para)))
print 'r_para mean: ' + str(np.mean(filter(lambda x: x!=0,r_para)))
print 'p_sen mean: ' + str(np.mean(filter(lambda x: x!=0,p_sen)))
print 'r_sen mean: ' + str(np.mean(filter(lambda x: x!=0,r_sen)))

p_para mean: 0.268214285714
r_para mean: 0.1875
p_sen mean: 0.11
r_sen mean: 0.117142857143
after filter:
p_para mean: 0.312916666667
r_para mean: 0.21875
p_sen mean: 0.1925
r_sen mean: 0.205


In [31]:
d = {'a':[1], 'b':[]}
dict(filter(lambda x: x[1].__len__() != 0, d.items()))

{'a': [1]}